# CBIP Study

In [9]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm, trange
from pyvis.network import Network
from coloring.CBIP import CBIP
from utils import generate_k_colorable_graph

ImportError: cannot import name 'CBIP' from 'coloring.CBIP' (c:\Users\pablo\OneDrive\Documents\Concordia\COMP 6651\project\github\COMP-6651-Project\src\coloring\CBIP.py)

### Specify parameters here

In [4]:
N = 50    # Number of online k-colorable graphs to generate at each step

In [6]:
# k, n, avg_comp_ratio, N
study_results = []
vertex_range = range(0,1500,100)[::2]
# chromatic number
for k in [2]:
    # number of vertices
    for n in tqdm(vertex_range):
        ratios = []
        for _ in range(N):
            # Generate a k-colorable graph
            G, _ = generate_k_colorable_graph(k, n, 0.7)
            # Color the graph
            G_ = CBIP(G)
            # Number of unqiue colors used
            num_colors = len(set([G_.nodes[i]['group'] for i in G_.nodes]))
            # Ratio of colors used to chromatic number
            ratio = num_colors / k
            ratios.append(ratio)
        
        # Average ratio of colors used to chromatic number
        avg_ratio = np.mean(ratios)
        study_results.append((k, n, avg_ratio, N))

        if n % 1000 == 0:
            df = pd.DataFrame(study_results, columns=['k', 'n', 'avg_comp_ratio', 'N'])
            df.to_feather(f'../data/CBIP_study_checkpoint.feather')


  0%|          | 0/8 [00:00<?, ?it/s]


NameError: name 'CBIP' is not defined

In [ ]:
df = pd.DataFrame(study_results, columns=['k', 'n', 'avg_comp_ratio', 'N'])
df.to_feather('../data/CBIP_study.feather')

In [3]:
# Create a graph
G = nx.Graph()
G.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (4, 5)])

# Check if the graph is bipartite
if nx.is_bipartite(G):
    # Find a bipartition of the graph
    bipartition = nx.bipartite.sets(G)

    # Get the two sets of nodes in the bipartition
    set1 = bipartition[0]
    set2 = bipartition[1]

    print("Set 1:", set1)
    print("Set 2:", set2)
else:
    print("The graph is not bipartite.")

Set 1: {1, 4}
Set 2: {2, 3, 5}
